<h1>Running and saving multiple simulations</h1>
<div class="text">
    In this notebook, I will explain how to create simulations of coeval cubes and save only the statistics of these coeval cubes.
    <br>
    Two functions are used to create these simulations:
     <ul>
      <li>psbi.create_batched_data_l1l2() which saves the $\ell^1$ and $\ell^2$ summaries, and the astrophysical paramters</li>
      <li>psbi.create_batched_data() which saves the statatistics PS, S1, S2, PS3d and the astrophysical parameters</li>
     </ul> 
</div>

In [1]:
import os
import sys
import numpy as np
sys.path.insert(1, os.path.abspath('../')) # Note that this line is useless with a regular pip installation of PyWST.
import pietrosbi_new as psbi

/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_cfg.py:57: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_cfg.py:41: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


<h2>Function for the statistics PS, S1, S2, PS3d</h2>

In [ ]:
# Number of bacthes for the simulations
n_batches = 2

# Number of simulation per batch
n_per_batch = 2

# So total number of simulations n = n_batches * n_per_batch

# Redshif, number of pixels and dimension of the cube
z = 9
n_pixels = 32
dim = 300

# NUmber of bins for the statistics
bins_list = [3,6]

# These three paramters are ignored since it doesn't calculate the ell summaries
l1 = True
l2 = True
wavelet_type = 'morl'

# Starting seed
start_seed = 1

# True: if you want to save the statistics. Remember to change the directory to the oneyou want
# False: if you don't want to save the statistics
save_files = True

# Choose the path. Remember to remove it from the comment.
## path = '/path/to/directory' ##

# If you want to anlayse the statistics, you have to first save the files
# The cleaning of the 21cmfast-cache is done inside the function
# The function also simulates one additional coeval cube which will be used as the 'observation' for the SBI
seed = psbi.create_batched_data(n_batches = n_batches, 
                                n_per_batch = n_per_batch, 
                                z = z, 
                                n_pixels = n_pixels, 
                                dim = dim, 
                                bins_list = bins_list, 
                                l1 = l1, 
                                l2 = l2, 
                                wavelet_type = wavelet_type, 
                                start_seed = start_seed, 
                                save_files = save_files,
                                path = path)


/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast

In [ ]:
# For loop for each number of bins
for b in bins_list:
    full_path = os.path.join(path, f'bins_{b}')
    
    # Loading and comabing the batches of the observed data
    data_obs_S1 = np.load(os.path.join(full_path, 'data_obs_S1.npy'))
    data_obs_PS = np.load(os.path.join(full_path, 'data_obs_PS.npy'))
    data_obs_PS3d = np.load(os.path.join(full_path, 'data_obs_PS3d.npy'))
    data_obs_S2 = np.load(os.path.join(full_path, 'data_obs_S2.npy'))
    param_true = np.load(os.path.join(full_path, 'param_true.npy'))[:2]
    
    # Loading and comabing the batches of the data of the simulations
    data_S1 = psbi.import_data_test(os.path.join(full_path, 'data_summary_S1_*.npy'))
    data_PS = psbi.import_data_test(os.path.join(full_path, 'data_summary_PS_*.npy'))
    data_PS3d = psbi.import_data_test(os.path.join(full_path, 'data_summary_PS3d_*.npy'))
    data_S2 = psbi.import_data_test(os.path.join(full_path, 'data_summary_S2_*.npy'))
    param_list = psbi.import_data_test(os.path.join(full_path, 'param_list_*.npy'))

    # Calculating the line of sight decomposition for PS, S1 and S2. 
    # These data are saved automatically in full_path in one single file for each statistic
    l1l2_summary_PS, l1l2_summary_S1, l1l2_summary_S2 = psbi.calculate_l1l2(data_PS, data_S1, data_S2, n_pixels, wavelet_type, l1, l2, full_path)
    l1l2_obs_summary_PS, l1l2_obs_summary_S1, l1l2_obs_summary_S2 = psbi.calculate_obs_l1l2(data_obs_PS, data_obs_S1, data_obs_S2, n_pixels, wavelet_type, l1, l2, full_path)

    # Saving the data of PS3d in one single file (before they were divided in batches)
    np.save(os.path.join(full_path, 'data_PS3d.npy'), data_PS3d)

    # Saving the parameters in one single file
    np.save(os.path.join(full_path, 'param_list.npy'), param_list)
    

<h2>Function for the $\ell^1$ and $\ell^2$ summaries</h2>

In [ ]:
# Number of bacthes for the simulations
n_batches = 2

# Number of simulation per batch
n_per_batch = 2

# So total number of simulations n = n_batches * n_per_batch

# Redshif, number of pixels and dimension of the cube
z = 9
n_pixels = 32
dim = 300

# NUmber of bins for the statistics
bins_list = [3, 6]

# True if you want the l1 summary, otherwise False
l1 = True

# True if you want the l2 summary, otherwise False
l2 = True

# Type wavelet for the line of sight decomposition
wavelet_type = 'morl'

# Starting seed
start_seed = 1

# True: if you want to save the statistics. Remember to change the directory to the oneyou want
# False: if you don't want to save the statistics
save_files = True

# Choose the path. Remember to remove it from the comment.
## path = '/path/to/directory' ## 

# If you want to anlayse the l1 and l2 summaries, you have to first save the files
# The cleaning of the 21cmfast-cache is done inside the function
# The function also simulates one additional coeval cube which will be used as the 'observation' for the SBI
seed = psbi.create_batched_data_l1l2(n_batches = n_batches, 
                              n_per_batch = n_per_batch, 
                              z = z, 
                              n_pixels = n_pixels, 
                              dim = dim, 
                              bins_list = bins_list, 
                              l1 = l1, 
                              l2 = l2, 
                              wavelet_type = wavelet_type, 
                              start_seed = start_seed, 
                              save_files = save_files,
                              path = path)


/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:400: UserWarning: The following parameters to FlagOptions are not supported: ['USE_VELS_AUX']
  warnings.warn(
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vx
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vy
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast/_utils.py:815: UserWarning: Trying to remove array that isn't yet created: hires_vz
  warnings.warn(f"Trying to remove array that isn't yet created: {k}")
/obs/pguidi/anaconda3/envs/sbi_21cmfast_env/lib/python3.9/site-packages/py21cmfast

In [ ]:
# Choose the path. Remember to remove it from the comment.
## path = '/path/to/directory' ##

bins_list = [3, 6]

# For loop for each number of bins
for b in bins_list:
    full_path = os.path.join(path, f'bins_{b}')
    
    # Loading and comabing the batches of the line of sight decomposition of PS, S1 and S2
    l1l2_summary_PS = psbi.import_data_l1l2(os.path.join(full_path, 'data_summary_PS_l1l2_*.npy'))
    l1l2_summary_S1 = psbi.import_data_l1l2(os.path.join(full_path, 'data_summary_S1_l1l2_*.npy'))
    l1l2_summary_S2 = psbi.import_data_l1l2(os.path.join(full_path, 'data_summary_S2_l1l2_*.npy'))

    # Loading and comabing the batches of the PS3d
    data_PS3d = psbi.import_data_test(os.path.join(full_path, 'data_summary_PS3d_*.npy'))

    # Loading and comabing the batches of the parameters
    param_list = psbi.import_data_test(os.path.join(full_path, 'param_list_*.npy'))
    
    # Saving the files of the statistics and parameters in one file instead of batches
    np.save(os.path.join(full_path, 'l1l2_summary_PS.npy'), l1l2_summary_PS)
    np.save(os.path.join(full_path, 'l1l2_summary_S1.npy'), l1l2_summary_S1)
    np.save(os.path.join(full_path, 'l1l2_summary_S2.npy'), l1l2_summary_S2)
    np.save(os.path.join(full_path, 'data_PS3d.npy'), data_PS3d)
    np.save(os.path.join(full_path, 'param_list.npy'), param_list)
    